## 文春オンラインの記事をスクレイピングして、ネガポジ分析を行います

### スクレイピングとは
スクレイピングを利用することでネット上の様々な情報を取得することができます。  
今回はPythonのコードを利用して記事を取得しています。  
`BeautifulSoup`などを利用して、HTMLやCSSの情報を指定して情報を抽出します。

### ネガポジ分析について
記事の内容がネガティブなのかポジティブなのかを[単語感情極性対応表](http://www.lr.pi.titech.ac.jp/~takamura/pndic_ja.html)を基準にして数値化してみようと思います。  
単語感情極性対応表は単語に対してネガポジ度を-1〜1で定義してあります。

喜び:よろこび:名詞:0.998861  
厳しい:きびしい:形容詞:-0.999755

などです。

### 参考文献
スクレイピングは[udemy講座](https://www.udemy.com/course/python-web-scraping-with-beautifulsoup-selenium-requests/)を利用しました。  
ネガポジ分析は[こちらの記事](https://qiita.com/g-k/items/e49f68d7e2fed6e300ea)を参考にさせていただきました。  
また、今回分析対象としたのは[文春オンライン](https://bunshun.jp/)です。  

では作業に取り掛かりましょう。  
まずは必要なライブラリをインポートします。

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd, numpy as np
import os
import glob
import pathlib
import re
import janome
import jaconv
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *

### まずはスクレイピングの準備です。
URLを取得して`requests`と`BeautifulSoup`を適用します。

In [2]:
url = "https://bunshun.jp/" # urlに文春オンラインのリンクを格納
res = requests.get(url) # requests.get()を用いてurlをresに格納
soup = BeautifulSoup(res.text, "html.parser") # ここでBeautifulSoupを用いてスクレイピングの準備ができました。

### 記事一覧を取得して、タイトルとURLを取得
基本的には記事が`li`要素として並んでおり、その親要素は`ul`であるというパターンが多いです。  
for文を使って記事一覧からタイトルとURLを取得しています。

In [3]:
elems = soup.select("ul") # 記事のリストがli要素として並んでいたので、その親要素であるulを指定しています。
title_list = [] # 記事のタイトルを格納するリスト
url_list = [] # 記事のURLを格納するリスト
for sibling in elems[3]: # elems[3]に欲しいリストがありました。このfor分により記事のリストから記事のタイトルとURLを取得し、それぞれリストに格納します。
    if sibling != "\n": # 改行が含まれていたので除外
        print(sibling.h3.string) # タイトルはh3タグに入っていました。
        title_list.append(sibling.h3.string.replace('\u3000', ' ')) # \u3000が入っている部分があったので空白に変換
        print(url + sibling.h3.a["href"]) # aタグのhref属性にリンクが格納されていました。
        url_list.append(url + sibling.h3.a["href"]) # 上記で取得したurl以下の部分が格納されていたので足しています。

波乱はルーキーたちの走り次第？　“怪物世代”の初陣に注目 第97回箱根駅伝予選会を読む
https://bunshun.jp//articles/-/40890
スマホやタブレットじゃやっぱり不十分？　「オンライン授業」に最適のPCとは
https://bunshun.jp//articles/-/39732
幼児教育「ヨコミネ式」創設者・横峯吉文氏のDVを元妻と子供が告白《法廷闘争“全面対決”へ》
https://bunshun.jp//articles/-/40817
BTS会社社長、韓国8位の株式富豪に…「共通点はオタクなこと」J.Y.Parkが語った素顔
https://bunshun.jp//articles/-/40886
「レジ袋はいりません」からの流れでびっくりするほど挙動不審になった話
https://bunshun.jp//articles/-/40849
三浦春馬さん、竹内結子さん、木村花さん…芸能人「自殺の連鎖」はなぜ起きるのか？
https://bunshun.jp//articles/-/40912
内部資料入手「GoToトラベル事務局」大手出向社員に日当4万円
https://bunshun.jp//articles/-/40879
「お前は何アゲだ？ 何で俺だけ“たたき上げ”なんだ！」“最後の怪物幹事長”二階俊博がキレた瞬間
https://bunshun.jp//articles/-/40895
「バラバラにした遺体を鍋で……」白石隆浩被告が証言した、おぞましい犯行の一部始終
https://bunshun.jp//articles/-/40913
「長さ」ではなく…美容師が教える、髪を切るときに言うといい「意外な言葉」
https://bunshun.jp//articles/-/40694


### 記事は1ページでは終わらない場合があります。1ページずつ遷移してリンクを取得します。
リンク一覧を作成するために、while文を作って次のページが表示されていればそのURLを取得してページに遷移し、  
遷移したページにも次のページへのリンクがあれば取得して遷移するというループを回します。  
こうすることにより、1タイトルのニュースに関する全てのページに関するリンク一覧を作成できます。

In [4]:
news_list = [] # 全てのニュース記事のリンクをここに格納します。
for pickup_link in url_list: # このfor文でURLリストからURLを取り出します。
    news = [] # ニュース記事はページごとに分かれているため、このリストに各ページのリンクを格納します。
    news.append(pickup_link) # 最初のリンクを格納
    pickup_res = requests.get(pickup_link) # requests.get()を用いてリンクからページを取得
    pickup_soup = BeautifulSoup(pickup_res.text, "html.parser") # BeautifulSoupを適用
    while True: # このwhile文では次のページへのリンクがあればそのリンクを取得し、そのページへ遷移するというループを回します。
        try: # 遷移した先で次のページへのリンクがあれば永遠にこのループを繰り返します。
            next_link = pickup_soup.find("a", class_="next menu-link ga_tracking")["href"] # next menu-link ga_trackingというクラスを持つaタグのhref属性が次のページへのリンクでした。
            next_link = url + next_link
            next_res = requests.get(next_link) # requests.get()とBeautifulSoupを用いて遷移先のページ情報を取得します。
            pickup_soup = BeautifulSoup(next_res.text, "html.parser")
            news.append(next_link) # newsに各ページ情報を追加します。
        except Exception: # 次のページへのリンクが無ければここの処理が行われます。
            news_list.append(news) # タイトル内の全ての記事をURLがnewsに格納されたのでそれをnews_listに格納します。
            break
display(news_list) # 作成したURLリストを表示します。

[['https://bunshun.jp//articles/-/40890',
  'https://bunshun.jp//articles/-/40890?page=2',
  'https://bunshun.jp//articles/-/40890?page=3'],
 ['https://bunshun.jp//articles/-/39732'],
 ['https://bunshun.jp//articles/-/40817',
  'https://bunshun.jp//articles/-/40817?page=2',
  'https://bunshun.jp//articles/-/40817?page=3',
  'https://bunshun.jp//articles/-/40817?page=4'],
 ['https://bunshun.jp//articles/-/40886',
  'https://bunshun.jp//articles/-/40886?page=2',
  'https://bunshun.jp//articles/-/40886?page=3'],
 ['https://bunshun.jp//articles/-/40849'],
 ['https://bunshun.jp//articles/-/40912',
  'https://bunshun.jp//articles/-/40912?page=2'],
 ['https://bunshun.jp//articles/-/40879',
  'https://bunshun.jp//articles/-/40879?page=2'],
 ['https://bunshun.jp//articles/-/40895',
  'https://bunshun.jp//articles/-/40895?page=2',
  'https://bunshun.jp//articles/-/40895?page=3'],
 ['https://bunshun.jp//articles/-/40913',
  'https://bunshun.jp//articles/-/40913?page=2',
  'https://bunshun.jp//art

### 先ほどのコードでURLリストを作成できたので、そのリンクを辿って記事本文を取得していきます。
`.text`を適用することで本文のみを取得できるのですが、細かくfor文を回して`.text`を適用しています。  
そのためいくつかの空文字(or空リスト)を作成して格納しながら本文を格納したリストを作成していきました。

In [5]:
news_page_list = [] # ここに全ての記事の本文を格納します。
for news_links in news_list: # このfor文でURLのリストからあるタイトルのリンクリストを取り出します。
    news_page = '' # ここに各ページから取得した本文を追加していきます。
    for news_link in news_links: # タイトルのリンクリストからリンクを一つずつ取り出します。
        news_res = requests.get(news_link) # requests.get()とBeautifulSoupを利用して記事の情報を取得します。
        news_soup = BeautifulSoup(news_res.text, "html.parser") 
        news_soup = news_soup.find(class_=re.compile("article-body")).find_all("p") # article-bodyというidを持つタグの直下のpタグに本文が格納されていました。
        news_phrase = '' # そのページの本文のフレーズを格納
        for news in news_soup: # for文で回すことでtextを適用して本文フレーズのみを取得できました。
            news_phrase += news.text.replace('\u3000', ' ') # 取得したフレーズを追加。文字列なので+で追加できました。
        news_page += news_phrase # 1ページ分のフレーズが取得できらたらnew_pageに追加
    news_page_list.append(news_page) # 一つのタイトルに対する全ての本文がnew_pageに格納されたらnews_page_listに追加。これはリスト型なのでappendを使用。

In [6]:
for i in range(1, 4): # 取得した本文の一部を表示してみます。うまく取得できたようです。
    print("<%s>" % i, news_page_list[i][:500], end="\n\n")

<1>  2020年は、まさにデジタル教育の歴史にとって特記されるべき重要な年となった。まずは、小学校においてプログラミング教育が必修化された。そして、予期せぬコロナ禍の到来によって、デジタル機器を使っての学習は不可避のものとなった。 コロナで家族そろって自宅に籠っていた間、親はリモートワーク、子どもはオンライン学習に励むため、家に1台しかないPCを親子で取り合うことになった経験を持つ読者は多いだろう。 また、最近では「BYOD」という略語が定着しつつある。「Bring your own device」を短縮したこのフレーズは、個人保有のPCを職場や学校に持ち込むことを表す。 今後、たとえコロナの感染が収束しようとも、リモートワークとオンライン学習、この新しいスタイルの普及する流れが、後退へと転じることは考えづらい。 だからこそ、ここで必要不可欠となってくるのが子ども専用のPCなのだ。 となると、「スマホやタブレットでいいんじゃないの？」という意見も出てくることだろう。しかし、やはりPCにはPCにしかない大きな強みがある。●キーボードがなくちゃ！ オンライン授業の際に、質問や発言といったチャッ

<2>  保育園児が跳び箱10段を跳び、倒立歩行。さらに九九算も覚え、漢字を読み書きし、3年間で平均2000冊の本を読破する――。「すべての子供が天才である」をモットーに、鹿児島から全国に広がった独創的な教育方法がある。子供の向上心や競争心に火を付け、強制することなく自分で学ぶ力を伸ばすこの教育法は「ヨコミネ式」と呼ばれている。いまでは、全国で約400もの保育園・幼稚園が、このヨコミネ式カリキュラムを導入しているほどの人気なのだ。 この教育法の考案者が元国会議員・横峯良郎氏の兄で、鹿児島で保育園理事長を務める横峯吉文氏（69）だ。 ヨコミネ式は20年ほど前に考案され、近年では、2019年の四大陸選手権で優勝した女子フィギュアスケーターの紀平梨花選手が「ヨコミネ式」を導入した幼稚園（兵庫県）の卒園生だったことから再び注目を集めた。 鹿児島県保育連合会理事の経歴を持つ吉文氏は、“幼児教育のカリスマ”として、講演会で全国を駆け回っている。独自の幼児教育論を唱える著作は、『「ヨコミネ式」子どもの才能の伸ばし方』（小学館）、『「ヨコミネ式」天才づくりの教科書』（講談社）など数十冊刊行

### スクレイピングによって得た、今までの情報を一つのDataFrameに格納します。
こうすることでデータが見やすくなるだけでなく、扱いやすくなります。  
ここまでできればあとはデータを加工してネガポジ分析をするだけです！

In [7]:
new_no_list = [x for x in range(len(title_list))] # あとで使うのでニュースNo.を作成
newslist = np.array([new_no_list, title_list, url_list, news_page_list]).T # DataFrameに格納する準備として、np.arrayのリストに格納して転置しておく。
newslist = pd.DataFrame(newslist, columns=['ニュースNo.', 'title', 'url', 'news_page_list']) # カラム名を指定してDataFrameに格納
newslist = newslist.astype({'ニュースNo.':'int64'}) # あとでテーブルを結合するためにニュースNo.をint64型に変換
newslist.to_csv('/work/data/BunshuOnline/news.csv')

In [8]:
display(newslist)

,ニュースNo.,title,url,news_page_list
0,0,波乱はルーキーたちの走り次第？ “怪物世代”の初陣に注目 第97回箱根駅伝予選会を読む,https://bunshun.jp//articles/-/40890,「例年なら『駅伝はトラックとは別物だから、1年生は戦力としてあまり期待しすぎないほうがいい』...
1,1,スマホやタブレットじゃやっぱり不十分？ 「オンライン授業」に最適のPCとは,https://bunshun.jp//articles/-/39732,2020年は、まさにデジタル教育の歴史にとって特記されるべき重要な年となった。まずは、小学...
2,2,幼児教育「ヨコミネ式」創設者・横峯吉文氏のDVを元妻と子供が告白《法廷闘争“全面対決”へ》,https://bunshun.jp//articles/-/40817,保育園児が跳び箱10段を跳び、倒立歩行。さらに九九算も覚え、漢字を読み書きし、3年間で平均...
3,3,BTS会社社長、韓国8位の株式富豪に…「共通点はオタクなこと」J.Y.Parkが語った素顔,https://bunshun.jp//articles/-/40886,上場前から大騒ぎだった。 10月15日、「防弾少年団（BTS）」の所属事務所「Big Hi...
4,4,「レジ袋はいりません」からの流れでびっくりするほど挙動不審になった話,https://bunshun.jp//articles/-/40849,漫画家の山本さほさんが、厄介な人たちを引き寄せるトラブル続きな日々をつづります。今回は、薬...
5,5,三浦春馬さん、竹内結子さん、木村花さん…芸能人「自殺の連鎖」はなぜ起きるのか？,https://bunshun.jp//articles/-/40912,三浦春馬さんが亡くなってから3カ月が経とうとしているが、いまだに彼の自死をめぐる報道は止ま...
6,6,内部資料入手「GoToトラベル事務局」大手出向社員に日当4万円,https://bunshun.jp//articles/-/40879,10月1日から東京発着の旅行も対象に加わった政府の観光支援策「GoToトラベル事業」。その...
7,7,「お前は何アゲだ？ 何で俺だけ“たたき上げ”なんだ！」“最後の怪物幹事長”二階俊博がキレた瞬間,https://bunshun.jp//articles/-/40895,自民党幹事長の通算在職記録を更新中の二階俊博は寝業師、すなわち「政治技術の巧者」という報道...
8,8,「バラバラにした遺体を鍋で……」白石隆浩被告が証言した、おぞましい犯行の一部始終,https://bunshun.jp//articles/-/40913,「背後に回って、首を左腕で絞めた」 2017年10月に神奈川県座間市のアパートで男女9人の遺...
9,9,「長さ」ではなく…美容師が教える、髪を切るときに言うといい「意外な言葉」,https://bunshun.jp//articles/-/40694,掛け布団がだんだん心地よくなってきた、今日この頃。朝夜の急激な気温差に、薄めの上着を羽織っ...


### ネガポジの判断基準には「単語感情極性対応表」を使用します。
この「[単語感情極性対応表](http://www.lr.pi.titech.ac.jp/~takamura/pndic_ja.html)」を分析で使用するための形に整えます。

In [9]:
p_dic = pathlib.Path('/work/data/dic') # workディレクトリのdicフォルダにパスを通します。ここに「単語感情極性対応表」のファイルを置いています。

for i in p_dic.glob('*.txt'): # 該当のファイルを見つけます。
    with open (i, 'r', encoding='utf-8') as f:
        x = [i.replace('\n', '').split(':') for i in f.readlines()] # 1行ずつ読み込みます。
        
posi_nega_df = pd.DataFrame(x, columns = ['基本形', '読み', '品詞', 'スコア']) # 読み込んだデータをDataFrameに格納します。
posi_nega_df['読み'] = posi_nega_df['読み'].apply(lambda x : jaconv.hira2kata(x)) # 平仮名をカタカナに変換(同じ読みのものが含まれており、重複を無くす為のようです。)
posi_nega_df = posi_nega_df[~posi_nega_df[['基本形', '読み', '品詞']].duplicated()] # 重複を削除します。
posi_nega_df.head()

,基本形,読み,品詞,スコア
0,優れる,スグレル,動詞,1
1,良い,ヨイ,形容詞,0.999995
2,喜ぶ,ヨロコブ,動詞,0.999979
3,褒める,ホメル,動詞,0.999979
4,めでたい,メデタイ,形容詞,0.999645


### 記事本文を形態素解析して分析に利用できる形にします。
形態素解析には`Tokenizer()`と`UnicodeNormalizeCharFilter()`を利用します。  
単語、基本形、品詞、読みを取り出してDataFrameに格納します。  
そして、記事のDataFrameと「単語感情極性対応表」をマージして、記事に含まれるワードをスコア化します。  
下記にその表を示しています。  
「人気」という単語が高いスコアを示していますのでポジティブな単語だと判断されました。  
他の単語はなぜそのスコアになったんだ？というものもありますが、気にせず進みましょう。

In [10]:
i = 0 # このiはニュースNo.を取得する際に利用します。

t = Tokenizer()
char_filters = [UnicodeNormalizeCharFilter()]
analyzer = Analyzer(char_filters=char_filters, tokenizer=t)

word_lists = []
for i, row in newslist.iterrows(): # iを一つずつ増やしていきニュースNo.とします。
    for t in analyzer.analyze(row[3]): # 取り出したレーベルの3カラム目に本文が格納されています。
        surf = t.surface # 単語
        base = t.base_form # 基本形
        pos = t.part_of_speech # 品詞
        reading = t.reading # 読み
        
        word_lists.append([i, surf, base, pos, reading]) # word_listsに追加
word_df = pd.DataFrame(word_lists, columns=['ニュースNo.', '単語', '基本形', '品詞', '読み'])
word_df['品詞'] = word_df['品詞'].apply(lambda x : x.split(',')[0]) # 品詞は複数格納されるが最初の1つのみ利用
display(word_df.head(10)) # 作成した本文のテーブルを表示
print("↓↓↓↓↓↓↓単語感情極性対応表とマージ↓↓↓↓↓↓↓")
score_result = pd.merge(word_df, posi_nega_df, on=['基本形', '品詞', '読み'], how='left') # 本文のテーブルと単語感情極性対応表をマージ
display(score_result.head(10)) # 作成したスコアテーブルを表示。「人気」という単語のスコアが高いのは分かるが他は微妙、、、

,ニュースNo.,単語,基本形,品詞,読み
0,0,「,「,記号,「
1,0,例年,例年,名詞,レイネン
2,0,なら,だ,助動詞,ナラ
3,0,『,『,記号,『
4,0,駅伝,駅伝,名詞,エキデン
5,0,は,は,助詞,ハ
6,0,トラック,トラック,名詞,トラック
7,0,と,と,助詞,ト
8,0,は,は,助詞,ハ
9,0,別物,別物,名詞,ベツモノ


↓↓↓↓↓↓↓単語感情極性対応表とマージ↓↓↓↓↓↓↓


,ニュースNo.,単語,基本形,品詞,読み,スコア
0,0,「,「,記号,「,NaN
1,0,例年,例年,名詞,レイネン,-0.37768
2,0,なら,だ,助動詞,ナラ,NaN
3,0,『,『,記号,『,NaN
4,0,駅伝,駅伝,名詞,エキデン,-0.467274
5,0,は,は,助詞,ハ,NaN
6,0,トラック,トラック,名詞,トラック,-0.48481
7,0,と,と,助詞,ト,NaN
8,0,は,は,助詞,ハ,NaN
9,0,別物,別物,名詞,ベツモノ,-0.365318


In [11]:
result = []
for i in range(len(score_result['ニュースNo.'].unique())): # ニュースNo.を利用してfor文を回します。
    temp_df = score_result[score_result['ニュースNo.']== i]
    text = ''.join(list(temp_df['単語'])) # 1タイトル無いの全ての単語をつなげる。
    score = temp_df['スコア'].astype(float).sum() # 1タイトル内のスコアを全て足し合わせる。➡︎累計スコア
    score_r = score/temp_df['スコア'].astype(float).count() # 本文の長さに影響されないように単語数で割り算する。➡︎標準化スコア
    result.append([i, text, score, score_r])

ranking = pd.DataFrame(result, columns=['ニュースNo.', 'テキスト', '累計スコア', '標準化スコア']).sort_values(by='標準化スコア', ascending=False).reset_index(drop=True) # 標準化スコアで並び替えてDataFrameに格納

In [12]:
ranking = pd.merge(ranking, newslist[['ニュースNo.', 'title', 'url']], on='ニュースNo.', how='left') # ニュースNo.基準でマージする。タイトルとURLを追加する。
ranking = ranking.reindex(columns=['ニュースNo.', 'title', 'url', 'テキスト', '累計スコア', '標準化スコア']) # カラムを並び替え

In [13]:
display(ranking)

,ニュースNo.,title,url,テキスト,累計スコア,標準化スコア
0,3,BTS会社社長、韓国8位の株式富豪に…「共通点はオタクなこと」J.Y.Parkが語った素顔,https://bunshun.jp//articles/-/40886,上場前から大騒ぎだった。 10月15日、「防弾少年団(BTS)」の所属事務所「Big Hit...,-143.216924,-0.374913
1,1,スマホやタブレットじゃやっぱり不十分？ 「オンライン授業」に最適のPCとは,https://bunshun.jp//articles/-/39732,2020年は、まさにデジタル教育の歴史にとって特記されるべき重要な年となった。まずは、小学校...,-166.408544,-0.379928
2,0,波乱はルーキーたちの走り次第？ “怪物世代”の初陣に注目 第97回箱根駅伝予選会を読む,https://bunshun.jp//articles/-/40890,「例年なら『駅伝はトラックとは別物だから、1年生は戦力としてあまり期待しすぎないほうがいい』...,-199.076451,-0.386556
3,4,「レジ袋はいりません」からの流れでびっくりするほど挙動不審になった話,https://bunshun.jp//articles/-/40849,漫画家の山本さほさんが、厄介な人たちを引き寄せるトラブル続きな日々をつづります。今回は、薬局...,-14.878369,-0.437599
4,9,「長さ」ではなく…美容師が教える、髪を切るときに言うといい「意外な言葉」,https://bunshun.jp//articles/-/40694,掛け布団がだんだん心地よくなってきた、今日この頃。朝夜の急激な気温差に、薄めの上着を羽織った...,-192.702889,-0.475810
5,2,幼児教育「ヨコミネ式」創設者・横峯吉文氏のDVを元妻と子供が告白《法廷闘争“全面対決”へ》,https://bunshun.jp//articles/-/40817,保育園児が跳び箱10段を跳び、倒立歩行。さらに九九算も覚え、漢字を読み書きし、3年間で平均2...,-316.137341,-0.478271
6,7,「お前は何アゲだ？ 何で俺だけ“たたき上げ”なんだ！」“最後の怪物幹事長”二階俊博がキレた瞬間,https://bunshun.jp//articles/-/40895,自民党幹事長の通算在職記録を更新中の二階俊博は寝業師、すなわち「政治技術の巧者」という報道の...,-183.426197,-0.481434
7,6,内部資料入手「GoToトラベル事務局」大手出向社員に日当4万円,https://bunshun.jp//articles/-/40879,10月1日から東京発着の旅行も対象に加わった政府の観光支援策「GoToトラベル事業」。その運...,-174.992867,-0.484745
8,8,「バラバラにした遺体を鍋で……」白石隆浩被告が証言した、おぞましい犯行の一部始終,https://bunshun.jp//articles/-/40913,「背後に回って、首を左腕で絞めた」 2017年10月に神奈川県座間市のアパートで男女9人の遺...,-195.168497,-0.534708
9,5,三浦春馬さん、竹内結子さん、木村花さん…芸能人「自殺の連鎖」はなぜ起きるのか？,https://bunshun.jp//articles/-/40912,三浦春馬さんが亡くなってから3カ月が経とうとしているが、いまだに彼の自死をめぐる報道は止まな...,-138.236658,-0.564231


In [14]:
print("<<ポジティブ1位>>", end="\n\n")
for i in range(1, 4):
    print(ranking.iloc[0, i])

<<ポジティブ1位>>

BTS会社社長、韓国8位の株式富豪に…「共通点はオタクなこと」J.Y.Parkが語った素顔
https://bunshun.jp//articles/-/40886
上場前から大騒ぎだった。 10月15日、「防弾少年団(BTS)」の所属事務所「Big Hit Entertainment(BHエンタテインメント、以下BH社)」がKOSPI(韓国取引所)に上場した。初値は公募価格の2.6倍をつけ、時価総額は8兆8169億ウォン(約8800億円同日15時時点)。韓国の3大音楽事務所の時価総額を合わせたものをはるかに超え、韓国最大手の音楽事務所に躍り出た。 創業者、パン・シヒョクCEO(最高経営責任者)も一気に株式富豪へひとっ飛び。エンターテインメント界最高の株式富豪となり、全体でも現代自動車グループ副会長を抑えて8位となった。ちなみに不動の1位はサムスン電子のイ・ゴンヒ会長だ。 今さら言うまでもないが、 BH社をここまで大きくしたのは、世界で人気のアーティスト「BTS」だ。 この8月に英語でリリースした新曲「Dynamite」では韓国人アーティストとして初めて米ビルボードチャートHOT100で1位を記録し、最近、配信したオンラインライブでは1億1400万回の応援クリックを集めるなど話題は尽きない。 韓国では、BTSはこの10年間で60兆ウォン(約5兆5000億円)、今年だけで6兆ウォン(約5500億円)の経済効果をもたらしたと伝えられている。 韓国では例えばオリンピックで金メダルを獲得するなど、スポーツで功績を残した選手は兵役を免除されるが、「BTS」もその大衆文化的功績から免除にすべしという発言が与党議員から出て、兵役に敏感な韓国では侃々諤々となった。その後、大衆文化・芸術的功績が認められる者には入隊延期が認められるよう兵役法改正が進められていることが発表され、もはや、いちアーティストを超えた存在になっている。 韓国の3大芸能事務所は、「2PM」や「TWICE」を生み出した「JYPエンタテインメント(JYP社)」、「BoA」や「東方神起」、「少女時代」を輩出した「SMエンタテインメント(SM社)」、「BIGBANG」、「BLACKPINK」が所属する「YGエンタテインメント(YG社)」といわれ、世界で活躍するK-POPアーティストのほとんどはこ

In [15]:
print("<<ネガティブ1位>>", end="\n\n")
for i in range(1, 4):
    print(ranking.iloc[-1, i])

<<ネガティブ1位>>

三浦春馬さん、竹内結子さん、木村花さん…芸能人「自殺の連鎖」はなぜ起きるのか？
https://bunshun.jp//articles/-/40912
三浦春馬さんが亡くなってから3カ月が経とうとしているが、いまだに彼の自死をめぐる報道は止まない。 三浦さんだけではない。リアリティー番組「テラスハウス」に出演していた木村花さん、芦名星さん、藤木孝さん、竹内結子さんなど、芸能人の死をめぐる報道が「次なる死」を招きかねないと警鐘を鳴らすのが、筑波大学教授の斎藤環氏だ。「大変言いにくいことですが、精神科で患者が自死すると、同じように死を選ぶ患者が続発することがあります。同様に、生と死の境界線上にいる人たちが、芸能人の死の『物語』に触れることで死に傾く可能性が非常に高い。 精神科医として私が診ている患者さんの中にも、三浦春馬さんや、竹内結子さんの自殺に深くショックを受けている方がいらっしゃいます。元からその芸能人の長年のファンだったわけではないのに、です。芸能人たちの死は、心が弱っている人たちに対して、大きな影響を及ぼしかねない。何とか立ち直っても、次の死でまたショックを受ける。メンタルを打ち砕くようなダブルパンチ、トリプルパンチを受けている人は、相当数いるはずです。このことを、きちんと考慮する必要があります」 警察庁の発表によれば、8月の自殺者は1854人で、前年同月比で251人増加している。今年上半期の自殺者数が前年を下回っていたことを考えれば、この夏に急増したと言っていい。自殺が増えた背景にあるのは、「コロナうつ」という言葉に象徴される、新型コロナウイルスがもたらした社会の閉塞感だ。 それにしても、なぜ華やかな世界にいる芸能人の訃報が続くのか。斎藤氏は、芸能人には「投影性同一視」と呼ばれる心理作用が起こりやすく、それが悪影響を及ぼしているのではないかと指摘する。「『投影性同一視』とは、例えば他人という鏡に自分の怒りを投影し、他人から自分が責められていると感ずるような心のメカニズムのことです。 普段は人気があり、テレビ出演すれば視聴率という数字や視聴者の反応でその人気を実感できる立場にあったのに、それを肌で感じる機会が、コロナ禍において奪われてしまった。自分が評価されているという確たる証拠がない状況においては、目の前の現実を良くも悪くも解釈で

最もポジティブな記事と最もネガティブな記事を表示させてみました。  
なんとなく正しく評価できている気がしますね！

説明は以上です。読んでいただいてありがとうございます。